# 🔑 groupByKey vs reduceByKey: Key-Value RDD Operations

**Time to complete:** 35 minutes  
**Difficulty:** Intermediate  
**Prerequisites:** RDD basics, key-value operations

---

## 🎯 Learning Objectives

By the end of this notebook, you will master:
- ✅ **`groupByKey()`** - Group values by key (wide transformation)
- ✅ **`reduceByKey()`** - Aggregate values by key efficiently
- ✅ **Performance differences** - Why reduceByKey is usually better
- ✅ **When to use each** - Choosing the right operation
- ✅ **Advanced patterns** - Complex aggregations

**This is one of the most important concepts for interview preparation!**

---

## 🔍 Understanding Key-Value RDDs

**Key-Value RDDs** are the foundation of distributed aggregations. Each element is a tuple `(key, value)`.

### Creating Key-Value RDDs:

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GroupByKey_vs_ReduceByKey") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

print(f"✅ Spark ready - Version: {spark.version}")

# Create key-value RDD
data = [
    ("Alice", 100),
    ("Bob", 200),
    ("Alice", 150),
    ("Charlie", 300),
    ("Bob", 50),
    ("Alice", 75)
]

kv_rdd = sc.parallelize(data)
print(f"Key-Value RDD: {kv_rdd.collect()}")
print(f"Is Pair RDD: {kv_rdd.map(lambda x: isinstance(x, tuple) and len(x) == 2).reduce(lambda a, b: a and b)}")

## 🗂️ groupByKey() Operation

**`groupByKey()` groups all values for each key into an iterable.**

### Characteristics:
- **Wide transformation** - Requires data shuffling across network
- **Returns**: `(key, [value1, value2, ...])` for each key
- **Memory intensive** - All values for a key must fit in memory
- **Lazy evaluation** - Doesn't execute until action called

### When to Use:
- When you need all values for a key (not just aggregated result)
- For complex operations that can't be done with simple aggregation
- When working with small datasets or few distinct keys

In [ ]:
# groupByKey example
grouped = kv_rdd.groupByKey()

print("📊 GROUPBYKEY OPERATION")
print("=" * 50)
print("Original data:")
for item in data:
    print(f"  {item}")

print("\ngroupByKey() result:")
for key, values_iterable in grouped.collect():
    values_list = list(values_iterable)
    print(f"  {key}: {values_list}")
    
# Manual aggregation after groupByKey
print("\nManual aggregation after groupByKey:")
for key, values_iterable in grouped.collect():
    values_list = list(values_iterable)
    total = sum(values_list)
    count = len(values_list)
    avg = total / count
    print(f"  {key}: sum={total}, count={count}, avg={avg:.1f}")

## ⚡ reduceByKey() Operation

**`reduceByKey(func)` aggregates values for each key using a reduction function.**

### Characteristics:
- **Wide transformation** - Also requires shuffling, but optimized
- **Returns**: `(key, aggregated_value)` for each key
- **Memory efficient** - Combines values locally before shuffling
- **Lazy evaluation** - Doesn't execute until action called

### When to Use:
- For simple aggregations (sum, count, min, max, etc.)
- When you only need the aggregated result (not individual values)
- For most aggregation use cases (preferred over groupByKey)

In [ ]:
# reduceByKey example
sum_by_key = kv_rdd.reduceByKey(lambda a, b: a + b)

print("⚡ REDUCEBYKEY OPERATION")
print("=" * 50)
print("Original data:")
for item in data:
    print(f"  {item}")

print("\nreduceByKey(lambda a, b: a + b) result:")
for key, total in sum_by_key.collect():
    print(f"  {key}: {total}")

# Multiple aggregations with reduceByKey
count_by_key = kv_rdd.mapValues(lambda v: 1).reduceByKey(lambda a, b: a + b)
max_by_key = kv_rdd.reduceByKey(lambda a, b: max(a, b))
min_by_key = kv_rdd.reduceByKey(lambda a, b: min(a, b))

print("\nAdvanced reduceByKey operations:")
print(f"Count: {dict(count_by_key.collect())}")
print(f"Max: {dict(max_by_key.collect())}")
print(f"Min: {dict(min_by_key.collect())}")

## 🚀 Performance Comparison

**Why reduceByKey is usually much faster than groupByKey!**

In [ ]:
# Create larger dataset for meaningful comparison
large_data = []
for i in range(10000):
    key = f"key_{i % 100}"  # 100 distinct keys
    value = i % 1000        # Values 0-999
    large_data.append((key, value))

large_rdd = sc.parallelize(large_data)
print(f"Large dataset: {len(large_data):,} elements, {len(set(k for k,v in large_data))} distinct keys")


In [ ]:
# Method 1: groupByKey + manual aggregation
print("=== Method 1: groupByKey + sum() ===")
start_time = time.time()

groupby_result = large_rdd.groupByKey().mapValues(lambda values: sum(values)).collect()
groupby_time = time.time() - start_time

print(f"groupByKey time: {groupby_time:.3f} seconds")
print(f"Results count: {len(groupby_result)}")

# Show first few results
print("Sample results:")
for key, total in sorted(groupby_result)[:5]:
    print(f"  {key}: {total}")
print(f"  ... and {len(groupby_result)-5} more")

In [ ]:
# Method 2: reduceByKey (efficient)
print("\n=== Method 2: reduceByKey ===")
start_time = time.time()

reducebykey_result = large_rdd.reduceByKey(lambda a, b: a + b).collect()
reducebykey_time = time.time() - start_time

print(f"reduceByKey time: {reducebykey_time:.3f} seconds")
print(f"Results count: {len(reducebykey_result)}")

# Show first few results
print("Sample results:")
for key, total in sorted(reducebykey_result)[:5]:
    print(f"  {key}: {total}")
print(f"  ... and {len(reducebykey_result)-5} more")

# Performance comparison
print(f"\n🎯 PERFORMANCE COMPARISON:")
print(f"groupByKey: {groupby_time:.3f}s")
print(f"reduceByKey: {reducebykey_time:.3f}s")
if groupby_time > 0:
    speedup = groupby_time / reducebykey_time
    print(f"reduceByKey is {speedup:.1f}x faster!")
else:
    print("reduceByKey is significantly faster!")

### 🔍 Why reduceByKey is Faster

**The key difference: Local aggregation before shuffling!**

```
groupByKey():
1. Shuffle ALL values for each key across network
2. Group them in memory on target executors
3. Apply aggregation function
❌ High network traffic, high memory usage

reduceByKey():
1. Pre-aggregate values locally on each executor
2. Shuffle only aggregated results
3. Final aggregation on target executors
✅ Minimal network traffic, efficient memory usage
```

**Network traffic reduction can be 10x-100x for large datasets!**

## 🎯 When to Use Each Operation

### Use `groupByKey()` when:
- ✅ You need **all individual values** for a key (not just aggregated result)
- ✅ You need **complex operations** that can't be done with simple reduction
- ✅ You have **few distinct keys** and small datasets
- ✅ You're doing **multiple different aggregations** on the same grouped data

### Use `reduceByKey()` when:
- ✅ You need **simple aggregations** (sum, count, min, max, etc.)
- ✅ You only need the **aggregated result** (not individual values)
- ✅ You have **many distinct keys** or large datasets
- ✅ **Performance matters** (which it usually does!)

In [ ]:
# Examples of when to use each

print("🎯 WHEN TO USE EACH OPERATION")
print("=" * 50)

# Example data: Student grades
grades_data = [
    ("Math", 85), ("Math", 92), ("Math", 78),
    ("Science", 88), ("Science", 95),
    ("English", 82), ("English", 79), ("English", 87)
]

grades_rdd = sc.parallelize(grades_data)

# Use reduceByKey for simple aggregation
print("reduceByKey - Simple aggregations:")
avg_grades = grades_rdd.mapValues(lambda grade: (grade, 1)) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .mapValues(lambda total_count: total_count[0] / total_count[1])

for subject, avg in avg_grades.collect():
    print(f"  {subject}: {avg:.1f}")

# Use groupByKey when you need all values
print("\ngroupByKey - Access to all values:")
grouped_grades = grades_rdd.groupByKey()

for subject, grade_iterable in grouped_grades.collect():
    grades = list(grade_iterable)
    print(f"  {subject}: {grades} (max: {max(grades)}, min: {min(grades)})")

# Use groupByKey for complex operations
print("\ngroupByKey - Complex analysis:")
complex_analysis = grades_rdd.groupByKey().mapValues(
    lambda grades: {
        'count': len(grades),
        'sum': sum(grades),
        'avg': sum(grades) / len(grades),
        'grades_above_85': len([g for g in grades if g >= 85])
    }
)

for subject, analysis in complex_analysis.collect():
    print(f"  {subject}: {analysis}")

## 🎯 Interview Questions & Key Takeaways

### Common Interview Questions:
1. **What's the difference between groupByKey and reduceByKey?**
2. **Why is reduceByKey more efficient?**
3. **When would you use groupByKey instead of reduceByKey?**
4. **How does data shuffling work in these operations?**
5. **What's the performance impact of choosing the wrong operation?**

### Answers:
- **groupByKey**: Groups all values for each key, returns iterators, requires shuffling all data
- **reduceByKey**: Aggregates values locally before shuffling, much more efficient
- **Use groupByKey**: When you need all individual values or complex operations
- **Shuffling**: Network data movement required for wide transformations
- **Performance**: reduceByKey can be 10-100x faster for large datasets

## 🧹 Cleanup

In [ ]:
# Note: Spark Session will be cleaned up automatically in Jupyter
# In production code, use: spark.stop()

print("🧹 Session cleanup complete")

print("\n📚 KEY TAKEAWAYS:")
print("- groupByKey: Use when you need all values for complex operations")
print("- reduceByKey: Use for efficient aggregations (preferred)")
print("- Performance: reduceByKey can be dramatically faster")
print("- Memory: groupByKey requires more memory for large datasets")
print("- Interviews: This topic comes up frequently!")